In [15]:
import sys
sys.path.append('../../../src')
import pandas
import random
import math
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, ProcessMode, VaryingProcess
from energiapy.components.location import Location
from energiapy.components.transport import Transport
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
# from energiapy.model.constraints.demand import constraint_demand2
from energiapy.components.result import Result
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.plot import plot_results, plot_scenario, plot_location
from energiapy.model.solve import solve
from pyomo.environ import Param
from energiapy.utils.scale_utils import scale_pyomo_set, scale_tuple, scale_list
import matplotlib.pyplot as plt
from matplotlib import rc
from typing import Union, Tuple, List
from pyomo.environ import ConcreteModel, Var

In [16]:
_time_intervals = 7  # Number of time intervals in a planning horizon    (L_chi)
_coms = 1
_exec_scenarios = 4  # Number of execution scenarios                     (chi)
M = 1e4  # Big M

# Define temporal scales
time_scales = TemporalScale(discretization_list=[_exec_scenarios, _time_intervals])
annualization_factor = 1/len(time_scales.scale[0])
scale_iter_list = time_scales.scale_iter(scale_level=len(time_scales.scale)-1)

In [17]:
def formulate_model(start_time:tuple, end_time:tuple, info_dict:dict=None):

    scales = TemporalScale(discretization_list=[_exec_scenarios, _time_intervals], start_zero=start_time, end_zero=end_time)
    
    loc6_varying_factor = pandas.DataFrame(data={'com1_process': [1]*len(scales.scale[0])})
    
    # ======================================================================================================================
    # Declare resources/commodities
    # ======================================================================================================================
    com1_pur = Resource(name='com1_pur', cons_max=75, block={'imp': 1, 'urg': 1}, price=0.00, label='Commodity 1 consumed from outside the system')
    
    com1_in = Resource(name='com1_in', label='Commodity 1 received')
    com1_out = Resource(name='com1_out', label='Commodity 1 to be sent out')
    
    com1_loc1_out = Resource(name='com1_loc1_out', label='Commodity 1 sent out from location 1')
    com1_loc2_out = Resource(name='com1_loc2_out', label='Commodity 1 sent out from location 2')
    com1_loc3_out = Resource(name='com1_loc3_out', label='Commodity 1 sent out from location 3')
    com1_loc4_out = Resource(name='com1_loc4_out', label='Commodity 1 sent out from location 4')
    com1_loc5_out = Resource(name='com1_loc5_out', label='Commodity 1 sent out from location 5')
    com1_loc6_out = Resource(name='com1_loc6_out', label='Commodity 1 sent out from location 6')
    com1_loc7_out = Resource(name='com1_loc7_out', label='Commodity 1 sent out from location 7')
    
    com1_sold = Resource(name='com1_sold', revenue=0.00, demand=True, sell=True, label='Commodity 1 sold to outside the system')
    
    # ======================================================================================================================
    # Declare processes/storage capacities
    # ======================================================================================================================
    com1_process_capacity = 125
    min_process_capacity = 0.01
    
    com1_procure = Process(name='procure com1', prod_max=125, conversion={com1_pur: -1, com1_in: 1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Procure com1')
    com1_sell = Process(name='sell com1', prod_max=com1_process_capacity, conversion={com1_out: -1, com1_sold: 1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Sell com1')
    
    com1_receive_loc1 = Process(name='com1_receive_loc1', prod_max=com1_process_capacity, conversion={com1_loc1_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Commodity 1 received from location 1')
    com1_receive_loc2 = Process(name='com1_receive_loc2', prod_max=com1_process_capacity, conversion={com1_loc2_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Commodity 1 received from location 2')
    com1_receive_loc3 = Process(name='com1_receive_loc3', prod_max=com1_process_capacity, conversion={com1_loc3_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Commodity 1 received from location 3')
    com1_receive_loc4 = Process(name='com1_receive_loc4', prod_max=com1_process_capacity, conversion={com1_loc4_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Commodity 1 received from location 4')
    com1_receive_loc5 = Process(name='com1_receive_loc5', prod_max=com1_process_capacity, conversion={com1_loc5_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Commodity 1 received from location 5')
    com1_receive_loc6 = Process(name='com1_receive_loc6', prod_max=com1_process_capacity, conversion={com1_loc6_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Commodity 1 received from location 6')
    com1_receive_loc7 = Process(name='com1_receive_loc7', prod_max=com1_process_capacity, conversion={com1_loc7_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Commodity 1 received from location 7')
    
    com1_process = Process(name='com1_process', prod_max=com1_process_capacity, conversion={com1_in: -1, com1_out: 1},  capex=0.01, vopex=0.01, prod_min=min_process_capacity, varying=[VaryingProcess.DETERMINISTIC_CAPACITY], label='Process the commodity through the location')
    
    # com1_process = Process(name='com1_process', prod_max=prod_max, conversion={0:{com1_in: -1, com1_out: 1}, 1:{com1_in: -1, com1_out: 1}, 2:{com1_in: -1, com1_out: 1}},  capex=0.01, vopex=0.01, prod_min=prod_min, rate_max=rate_max, varying=[VaryingProcess.DETERMINISTIC_CAPACITY], label='Process the commodity through the location')
    
    com1_store10 = Process(name='com1_store10', prod_max=com1_process_capacity, capex=100, vopex=1, store_min=0.01, store_max= 20, prod_min=min_process_capacity, label="Storage capacity of 10 units", storage=com1_in, storage_cost=0.02)
    com1_store20 = Process(name='com1_store20', prod_max=com1_process_capacity, capex=200, vopex=2, store_min=0.01,store_max= 40, prod_min=min_process_capacity, label="Storage capacity of 20 units", storage=com1_in, storage_cost=0.02)
    com1_store50 = Process(name='com1_store50', prod_max=com1_process_capacity, capex=500, vopex=5, store_min=0.01, store_max= 100, prod_min=min_process_capacity, label="Storage capacity of 50 units", storage=com1_in, storage_cost=0.02)
    
    com1_loc1_send = Process(name='com1_loc1_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc1_out:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Send commodity one from location 1')
    com1_loc2_send = Process(name='com1_loc2_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc2_out:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Send commodity one from location 2')
    com1_loc3_send = Process(name='com1_loc3_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc3_out:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Send commodity one from location 3')
    com1_loc4_send = Process(name='com1_loc4_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc4_out:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Send commodity one from location 4')
    com1_loc5_send = Process(name='com1_loc5_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc5_out:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Send commodity one from location 5')
    com1_loc6_send = Process(name='com1_loc6_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc6_out:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Send commodity one from location 6')
    com1_loc7_send = Process(name='com1_loc7_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc7_out:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Send commodity one from location 7')
    
    # ======================================================================================================================
    # Declare locations/warehouses
    # ======================================================================================================================
    loc1 = Location(name='loc1', processes={com1_procure, com1_receive_loc2, com1_receive_loc3, com1_process, com1_store20, com1_loc1_send}, label="Location 1", scales=scales, demand_scale_level=1, capacity_scale_level=0, availability_scale_level=0)
    
    loc2 = Location(name='loc2', processes={com1_receive_loc1, com1_receive_loc4, com1_receive_loc5, com1_process, com1_store20, com1_loc2_send}, label="Location 2", scales=scales, demand_scale_level=1, capacity_scale_level=0, availability_scale_level=0)
    
    loc3 = Location(name='loc3', processes={com1_receive_loc1, com1_receive_loc4, com1_process, com1_store20, com1_loc3_send}, label="Location 3", scales=scales, demand_scale_level=1, capacity_scale_level=0, availability_scale_level=0)
    
    loc4 = Location(name='loc4', processes={com1_receive_loc2, com1_receive_loc3, com1_receive_loc6, com1_receive_loc5, com1_receive_loc7, com1_process, com1_store50, com1_loc4_send}, label="Location 4", scales=scales, demand_scale_level=1, capacity_scale_level=0, availability_scale_level=0)
    
    loc5 = Location(name='loc5', processes={com1_receive_loc2, com1_receive_loc4, com1_receive_loc7, com1_process, com1_store50, com1_loc5_send, com1_sell}, label="Location 5", scales=scales, demand_scale_level=1, capacity_scale_level=0, availability_scale_level=0)
    
    loc6 = Location(name='loc6', processes={com1_procure, com1_receive_loc4, com1_process, com1_store10, com1_loc6_send}, label="Location 6", scales=scales, demand_scale_level=1, capacity_scale_level=0, availability_scale_level=0, capacity_factor={com1_process: loc6_varying_factor[['com1_process']]})
    
    loc7 = Location(name='loc7', processes={com1_receive_loc4, com1_receive_loc5, com1_process, com1_store20, com1_loc7_send}, label="Location 7", scales=scales, demand_scale_level=1, capacity_scale_level=0, availability_scale_level=0)
    
    # ======================================================================================================================
    # Declare transport/trucks
    # ======================================================================================================================
    
    truck_cap12 = 70
    truck_cap13 = 30
    truck_cap24 = 50
    truck_cap25 = 30
    truck_cap34 = 30
    truck_cap45 = 100
    truck_cap47 = 40
    truck_cap64 = 50
    truck_cap75 = 40
    
    truck_capmin = 0.01
    
    truck12 = Transport(name='truck12', resources={com1_loc1_out}, trans_max=truck_cap12, label='Truck from location 1 to 2', capex=0.5, vopex=0.05, trans_min=truck_capmin, speed=50)
    truck21 = Transport(name='truck21', resources={com1_loc2_out}, trans_max=truck_cap12, label='Truck from location 2 to 1', capex=0.0001, vopex=0.05, trans_min=truck_capmin, speed=50)
    
    truck13 = Transport(name='truck13', resources={com1_loc1_out}, trans_max=truck_cap13, label='Truck from location 1 to 3', capex=0.3, vopex=0.03, trans_min=truck_capmin, speed=50)
    truck31 = Transport(name='truck31', resources={com1_loc3_out}, trans_max=truck_cap13, label='Truck from location 3 to 1', capex=0.0001, vopex=0.03, trans_min=truck_capmin, speed=50)
    
    truck24 = Transport(name='truck24', resources={com1_loc2_out}, trans_max=truck_cap24, label='Truck from location 2 to 4', capex=0.5, vopex=0.05, trans_min=truck_capmin, speed=50)
    truck42 = Transport(name='truck42', resources={com1_loc4_out}, trans_max=truck_cap24, label='Truck from location 4 to 2', capex=0.0001, vopex=0.05, trans_min=truck_capmin, speed=50)
    
    truck25 = Transport(name='truck25', resources={com1_loc2_out}, trans_max=truck_cap25, label='Truck from location 2 to 5', capex=0.3, vopex=0.03, trans_min=truck_capmin, speed=50)
    truck52 = Transport(name='truck52', resources={com1_loc5_out}, trans_max=truck_cap25, label='Truck from location 5 to 2', capex=0.0001, vopex=0.03, trans_min=truck_capmin, speed=50)
    
    truck34 = Transport(name='truck34', resources={com1_loc3_out}, trans_max=truck_cap34, label='Truck from location 3 to 4', capex=0.2, vopex=0.02, trans_min=truck_capmin, speed=50)
    truck43 = Transport(name='truck43', resources={com1_loc4_out}, trans_max=truck_cap34, label='Truck from location 4 to 3', capex=0.0001, vopex=0.02, trans_min=truck_capmin, speed=50)
    
    truck45 = Transport(name='truck45', resources={com1_loc4_out}, trans_max=truck_cap45, label='Truck from location 4 to 5', capex=1, vopex=0.1, trans_min=truck_capmin, speed=50)
    truck54 = Transport(name='truck54', resources={com1_loc5_out}, trans_max=truck_cap45, label='Truck from location 5 to 4', capex=0.0001, vopex=0.1, trans_min=truck_capmin, speed=50)
    
    truck47 = Transport(name='truck47', resources={com1_loc4_out}, trans_max=truck_cap47, label='Truck from location 4 to 7', capex=0.4, vopex=0.04, trans_min=truck_capmin, speed=50)
    truck74 = Transport(name='truck74', resources={com1_loc7_out}, trans_max=truck_cap47, label='Truck from location 7 to 4', capex=0.0001, vopex=0.04, trans_min=truck_capmin, speed=50)
    
    truck64 = Transport(name='truck64', resources={com1_loc6_out}, trans_max=truck_cap64, label='Truck from location 6 to 4', capex=0.5, vopex=0.05, trans_min=truck_capmin, speed=50)
    truck46 = Transport(name='truck46', resources={com1_loc4_out}, trans_max=truck_cap64, label='Truck from location 4 to 6', capex=0.0001, vopex=0.05, trans_min=truck_capmin, speed=50)
    
    truck75 = Transport(name='truck75', resources={com1_loc7_out}, trans_max=truck_cap75, label='Truck from location 7 to 5', capex=0.4, vopex=0.04, trans_min=truck_capmin, speed=50)
    truck57 = Transport(name='truck57', resources={com1_loc5_out}, trans_max=truck_cap75, label='Truck from location 5 to 7', capex=0.0001, vopex=0.04, trans_min=truck_capmin, speed=50)
    
    # ======================================================================================================================
    # Declare network
    # ======================================================================================================================
    
    # transport_matrix = [
    #     [[], [truck50_12], [truck30_13], [], [], [], []],  # source: location 1
    #     [[truck50_21], [], [], [truck50_24], [truck30_25], [], []],  # source: location 2
    #     [[truck30_31], [], [], [truck20_34], [], [], []],  # source: location 3
    #     [[], [truck50_42], [truck20_43], [], [truck100_45], [truck50_46], [truck40_47]],  # source: location 4
    #     [[], [truck30_52], [], [truck100_54], [], [], [truck40_57]],  # source: location 5
    #     [[], [], [], [truck50_64], [], [], []],  # source: location 6
    #     [[], [], [], [truck40_74], [truck40_75], [], []]  # source: location 7
    # ]
    
    transport_matrix = [
        [[], [truck12], [truck13], [], [], [], []],  # source: location 1
        [[truck21], [], [], [truck24], [truck25], [], []],  # source: location 2
        [[truck31], [], [], [truck34], [], [], []],  # source: location 3
        [[], [truck42], [truck43], [], [truck45], [truck46], [truck47]],  # source: location 4
        [[], [truck52], [], [truck54], [], [], [truck57]],  # source: location 5
        [[], [], [], [truck64], [], [], []],  # source: location 6
        [[], [], [], [truck74], [truck75], [], []]  # source: location 7
    ]
    
    distance_matrix = [
        [0, 55, 196, M, M, M, M],
        [55, 0, M, 163, 112, M, 134],
        [196, M, 0, 63, M, M, M],
        [M, 163, 63, 0, 95, 117, 88],
        [M, 112, M, 95, 0, M, 134],
        [M, M, M, 117, M, 0, M],
        [M, 134, M, 88, 134, M, 0]
    ]
    
    locset = [loc1, loc2, loc3, loc4, loc5, loc6, loc7]
    
    sources = locset
    sinks = locset
    
    network = Network(name='Network', scales= scales, source_locations=sources, sink_locations=sinks, transport_matrix=transport_matrix, distance_matrix=distance_matrix)
    
    # ======================================================================================================================
    # Declare scenario
    # ======================================================================================================================
    
    daily_demand = 100
    demand_penalty = 50
    
    demand_dict = {i: {com1_sold: daily_demand} if i == loc5 else {com1_sold: 0} for i in locset}
    demand_penalty_dict = {i: {com1_sold: demand_penalty} if i == loc5 else {com1_sold: 0} for i in locset}
    
    scenario = Scenario(name=f'scenario', scales=scales, scheduling_scale_level=1, network_scale_level=0, purchase_scale_level=1, availability_scale_level=0, demand_scale_level=1, capacity_scale_level=0, network=network, demand=demand_dict, demand_penalty=demand_penalty_dict, label='Baseline Scenario', annualization_factor=annualization_factor)
    
    # ======================================================================================================================
    # Declare model
    # ======================================================================================================================
    
    problem_mincost = formulate(scenario=scenario,
                            constraints={Constraints.COST, Constraints.TRANSPORT, Constraints.RESOURCE_BALANCE, Constraints.INVENTORY, Constraints.PRODUCTION, Constraints.DEMAND, Constraints.MODE, Constraints.NETWORK},
                            demand_sign='eq', objective=Objective.COST_W_DEMAND_PENALTY)
    
    return scenario, problem_mincost

In [18]:
def fix_variables(model2fix: ConcreteModel, current_time_idx:tuple, initial_dict: dict,):
    model2fix_vars = {v.name: v for v in model2fix.component_objects(Var)}
    
    for name, var in model2fix_vars.items():
        var_key_list = [k for k in var.keys()]
        updated_key_list = list()
        
        if var_key_list and var_key_list!= [None]:
            if isinstance(var_key_list[0], int):
                updated_key_list = var_key_list if tuple(var_key_list) < current_time_idx[:1] else []
            elif isinstance(var_key_list[0], tuple):
                num_integers = sum(1 for elem in var_key_list[0] if isinstance(elem, int))
                scale_length = min(len(time_scales.scale), num_integers)
                
                updated_key_list = [k for k in var.keys()
                                    if tuple(k[-scale_length:]) < current_time_idx[:scale_length] or (tuple(k[-scale_length:]) == current_time_idx[:scale_length] and current_time_idx[1:]!=(0,)*(len(current_time_idx)-1) and tuple(k[-scale_length:]) != current_time_idx and (name in ['Cap_F', 'Cap_P', 'Cap_S', 'X_F', 'X_P', 'X_S']))]                   
                
            for key in updated_key_list:
                if key in initial_dict[name] and initial_dict[name][key] is not None:
                    var[key].fixed = True
                    var[key] = initial_dict[name][key]
    print(f"Variables fixed for time {current_time_idx}")

In [19]:
def print_fixed_variables(instance:ConcreteModel):    
    for v in instance.component_objects(Var):
        for index in v:
            if v[index].fixed:
                print(f"{v.name}[{index}]: {v[index].value}")

def get_fixed_variables(instance:ConcreteModel):
    return {v.name: {index: v[index].value for index in v if v[index].fixed} for v in instance.component_objects(Var)}


In [20]:
# def compile_results(close_loop_results:dict=None, time_dict:dict=None, start:int=None, end:int=None):
#     final_results = {key: {} for key in close_loop_results[time_dict[start]]}
#     for t in range(start,end+1):
#         for name in close_loop_results[time_dict[t]]:
#             for key, value in name.items():
#                 if key:
#                     if isinstance(key, int):
#                         
#                 num_integers = sum(1 for elem in key)
#     
#     return final_results    

In [21]:
model_record = dict()
scenario_record = dict()
results_record = dict()

In [22]:
def supply_chain_controller(time_ind:List, start_at:Union[int, Tuple]=0, end_at:Union[int, Tuple]=None, initialize_dict:dict=None, p_horizon:int=1, c_horizon:int=1, lookback:int=0):
    
    end_at = end_at or len(time_ind)-1
    
    close_loop_results = dict()
    if c_horizon > p_horizon:
        raise ValueError('c_horizon must be less than or equal to p_horizon')
    
    if isinstance(start_at, Tuple):
        if start_at not in time_ind:
            raise ValueError('Incorrect start time value')
        start_idx = time_ind.index(start_at)
    else:
        start_idx = start_at
        
    if isinstance(end_at, Tuple):
        if end_at not in time_ind:
            raise ValueError('Incorrect end time value')
        end_idx = time_ind.index(end_at)
    else:
        end_idx = end_at
    
    if end_idx < start_idx:
        raise ValueError('Incorrect start and end time values')

    # print(f"Starting Index: {start_idx}; Starting Time: {start_time}")
    # print(f"Ending Index: {end_idx}; Ending Time: {end_time}")
    
    
    for t in range(start_idx, end_idx+1, c_horizon):
        start_time = max(0, t-lookback)
        end_time = min(t+p_horizon, end_idx)
        scenario, model = formulate_model(start_time=time_ind[start_time], end_time=time_ind[end_time])
        scenario_record[time_ind[t]] = scenario
        model_record[time_ind[t]] = model
        
        if time_ind[t-1] in close_loop_results:
            fix_variables(model2fix=model, initial_dict=close_loop_results[time_ind[t-1]], current_time_idx=time_ind[t])
        
        result = solve(scenario=scenario, instance=model, solver='gurobi', name='MILP')
        model_vars = model.component_map(ctype=Var)
        close_loop_results[time_ind[t]] = {i: model_vars[i].extract_values() for i in model_vars.keys()}
        results_record[time_ind[t]] = result

        print(f'\n################################## Finished solving problem for time period {time_ind[t]} out of {time_ind[-1]} ##################################\n')
        
    # final_results = compile_results(close_loop_results=close_loop_results, time_dict = time_ind, start = start_idx, end = end_idx)
    
    return close_loop_results

In [23]:
close_loop_results = supply_chain_controller(time_ind=scale_iter_list, c_horizon=1, p_horizon=5, lookback=10, end_at=(1,4))

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint location fopex
constraint storage cost
constraint storage capex
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constraint inventory network
constraint resource export
constraint transport export
constraint export
constraint transport capex
constraint transport network capex
constraint transport export network
constraint transport vopex
constraint transport network vopex
constraint transport fopex
constraint transport network fopex
constraint transport capacity UB no bin
constraint transport capacity LB no bin
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint transport capacity UB
constraint transport capacity LB
constraint fixed nameplate min production mode
constraint fixed nameplate max

## Fix planning horizon for a design problem

In [28]:
model_record

{(0, 0): <pyomo.core.base.PyomoModel.ConcreteModel at 0x21624b0fde0>,
 (0, 1): <pyomo.core.base.PyomoModel.ConcreteModel at 0x215c8f81860>,
 (0, 2): <pyomo.core.base.PyomoModel.ConcreteModel at 0x215c902db80>,
 (0, 3): <pyomo.core.base.PyomoModel.ConcreteModel at 0x216316c3660>,
 (0, 4): <pyomo.core.base.PyomoModel.ConcreteModel at 0x21637653980>,
 (0, 5): <pyomo.core.base.PyomoModel.ConcreteModel at 0x2163d035b30>,
 (0, 6): <pyomo.core.base.PyomoModel.ConcreteModel at 0x216428d7200>,
 (1, 0): <pyomo.core.base.PyomoModel.ConcreteModel at 0x2164a4f3840>,
 (1, 1): <pyomo.core.base.PyomoModel.ConcreteModel at 0x2164fa577a0>,
 (1, 2): <pyomo.core.base.PyomoModel.ConcreteModel at 0x216435500f0>,
 (1, 3): <pyomo.core.base.PyomoModel.ConcreteModel at 0x2165b079310>,
 (1, 4): <pyomo.core.base.PyomoModel.ConcreteModel at 0x2165c92fc00>}

In [30]:
model_record[(0,0)].Fopex_transport.pprint()

Fopex_transport : fopex of transport mode between sink and source
    Size=882, Index=sources*sinks*transports*scales_network
    Key                            : Lower : Value : Upper : Fixed : Stale : Domain
    ('loc1', 'loc1', 'truck12', 0) :     0 :   0.0 :  None : False :  True : NonNegativeReals
    ('loc1', 'loc1', 'truck13', 0) :     0 :   0.0 :  None : False :  True : NonNegativeReals
    ('loc1', 'loc1', 'truck21', 0) :     0 :   0.0 :  None : False :  True : NonNegativeReals
    ('loc1', 'loc1', 'truck24', 0) :     0 :   0.0 :  None : False :  True : NonNegativeReals
    ('loc1', 'loc1', 'truck25', 0) :     0 :   0.0 :  None : False :  True : NonNegativeReals
    ('loc1', 'loc1', 'truck31', 0) :     0 :   0.0 :  None : False :  True : NonNegativeReals
    ('loc1', 'loc1', 'truck34', 0) :     0 :   0.0 :  None : False :  True : NonNegativeReals
    ('loc1', 'loc1', 'truck42', 0) :     0 :   0.0 :  None : False :  True : NonNegativeReals
    ('loc1', 'loc1', 'truck43', 0) :  

In [29]:
print_fixed_variables(model_record[(1,0)])

P[('loc1', 'com1_loc1_send', 0, 0)]: 30.0
P[('loc1', 'com1_loc1_send', 0, 1)]: 30.0
P[('loc1', 'com1_loc1_send', 0, 2)]: 30.0
P[('loc1', 'com1_loc1_send', 0, 3)]: 30.0
P[('loc1', 'com1_loc1_send', 0, 4)]: 30.0
P[('loc1', 'com1_loc1_send', 0, 5)]: 30.0
P[('loc1', 'com1_loc1_send', 0, 6)]: 30.0
P[('loc1', 'com1_loc2_send', 0, 0)]: 0.0
P[('loc1', 'com1_loc2_send', 0, 1)]: 0.0
P[('loc1', 'com1_loc2_send', 0, 2)]: 0.0
P[('loc1', 'com1_loc2_send', 0, 3)]: 0.0
P[('loc1', 'com1_loc2_send', 0, 4)]: 0.0
P[('loc1', 'com1_loc2_send', 0, 5)]: 0.0
P[('loc1', 'com1_loc2_send', 0, 6)]: 0.0
P[('loc1', 'com1_loc3_send', 0, 0)]: 0.0
P[('loc1', 'com1_loc3_send', 0, 1)]: 0.0
P[('loc1', 'com1_loc3_send', 0, 2)]: 0.0
P[('loc1', 'com1_loc3_send', 0, 3)]: 0.0
P[('loc1', 'com1_loc3_send', 0, 4)]: 0.0
P[('loc1', 'com1_loc3_send', 0, 5)]: 0.0
P[('loc1', 'com1_loc3_send', 0, 6)]: 0.0
P[('loc1', 'com1_loc4_send', 0, 0)]: 0.0
P[('loc1', 'com1_loc4_send', 0, 1)]: 0.0
P[('loc1', 'com1_loc4_send', 0, 2)]: 0.0
P[('loc1'

In [26]:
model_record[(1,3)].scales_scheduling.pprint()

scales_scheduling : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     2 :    Any :   12 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4)}


In [27]:
model_record[(1,4)].scales_scheduling.pprint()

scales_scheduling : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     2 :    Any :   11 : {(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4)}
